### An AI agent that can refactor an entire codebase to improve the quality of written code

In [12]:
# Imports

import os
from pathlib import Path
from typing import List, Optional
from openai import OpenAI
from dotenv import load_dotenv

In [13]:
# Get API key

load_dotenv(override=True)
google_api_key = os.getenv("GOOGLE_API_KEY")

if google_api_key:
    print(f"Google API key start with '{google_api_key[:4]}...'")
else:
    print("Google API key not found")

Google API key start with 'AIza...'


In [14]:
# Constants

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
repo_root = Path("/Users/huzaifa/projects/ai-agent-code-polisher/unoptimized-code")
output_root = Path("/Users/huzaifa/projects/ai-agent-code-polisher/polished-code")

In [15]:
# File extensions to be processed
EXTENSIONS = {
    ".py": "python",
    ".js": "javascript",
    ".ts": "typescript",
    ".jsx": "javascript",
    ".tsx": "typescript (react)",
    ".java": "java",
    ".kt": "kotlin",
    ".swift": "swift",
}

In [16]:
system_prompt = """
You are a senior software engineer with 15+ years of experience writing and reviewing production code in multiple languages, including Python, JavaScript, TypeScript, React, Java, Kotlin, and Swift.

Your goals when rewriting code are:
- Improve clarity and maintainability.
- Preserve behavior exactly unless explicitly told otherwise.
- Apply idiomatic patterns for the given language and ecosystem.
- Make the code easier to test and debug.
- Don't include the triple back ticks that specifies the language type

General guidelines:
- Prefer clear, simple solutions over clever tricks.
- Use good naming: descriptive, consistent, and intent-revealing.
- Keep functions and methods small and focused on a single responsibility.
- Reduce duplication where it is safe to do so.
- Improve structure and organization (e.g. extract helpers, separate concerns).
- Add or improve docstrings and comments when they add important context (the “why”), but do not over-comment obvious code.
- Use consistent formatting and style conventions appropriate for the language (e.g. PEP8 for Python, modern ES/TypeScript style for JS/TS, idiomatic patterns for React, Java, Kotlin, Swift).
- Add basic error handling and input validation where it is obviously missing and important.
- Avoid introducing new dependencies unless required.
- Do not change public APIs, data models, or external behavior unless it is clearly a bug and the fix is obvious.
- Preserve all TODO comments and important notes from the original code.

Always return only the rewritten code, in the same language and with compatible structure, so it can drop in as a replacement.
"""

In [17]:
def call_llm(system_prompt: str, user_prompt: str) -> str:
    """
    Call your chosen LLM here and return the model's text output.
    This is a placeholder – integrate with OpenAI, local Llama, etc.
    """
    
    gemini = OpenAI(base_url=gemini_url, api_key=google_api_key)
    resp = gemini.chat.completions.create(
        model="gemini-3-pro-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    return resp.choices[0].message.content


In [18]:
def find_source_files(root: Path, exts: dict) -> List[Path]:
    paths: List[Path] = []
    for dirpath, _, filenames in os.walk(root):
        # Skip certain directories to avoid noise
        if ".git" in dirpath or "__pycache__" in dirpath or "node_modules" in dirpath:
            continue
        for name in filenames:
            p = Path(dirpath) / name
            if p.suffix in exts:
                paths.append(p)
    return paths


def build_user_prompt(code: str, language_label: str) -> str:
    return f"""
Rewrite the following {language_label} code to be clearer, more maintainable, and idiomatic, following the senior engineering guidelines.

Constraints:
- Preserve all external behavior.
- Keep public APIs and function signatures compatible unless there is a clear bug.
- Keep comments that provide important context or TODOs, but remove redundant or outdated comments.
- If you extract helper functions or classes, define them in the same file.
- If you see obvious dead code, you may safely remove it.

Here is the file content:

```{language_label}
{code}
"""

In [19]:
# Processes and rewrites each file

def process_file(path: Path) -> Optional[str]:
    ext = path.suffix
    if ext not in EXTENSIONS:
        return None

    language_label = EXTENSIONS[ext]
    original_code = path.read_text(encoding="utf-8")

    user_prompt = build_user_prompt(original_code, language_label)

    print(f"Rewriting {path}...")
    rewritten_code = call_llm(system_prompt, user_prompt)

    return rewritten_code

In [ ]:
def relative_output_path(src_path: Path) -> Path:
    rel = src_path.relative_to(repo_root)
    out_path = output_root / rel
    #out_path.parent.mkdir(parents=True, exist_ok=True)
    return out_path


def rewrite_repo():
    files = find_source_files(repo_root, EXTENSIONS)
    print(f"Found {len(files)} source files to consider.")

    for f in files:
        try:
            new_code = process_file(f)
            if new_code is None:
                continue

            out_path = relative_output_path(f)
            out_path.write_text(new_code, encoding="utf-8")
            print(f"✅ Rewritten {out_path}")
        except Exception as e:
            print(f"❌ Error processing {f}: {e}")

In [21]:
rewrite_repo()

Found 2 source files to consider.
Rewriting /Users/huzaifa/projects/ai-agent-code-polisher/unoptimized-code/sample_two.js...
✅ Wrote rewritten file to /Users/huzaifa/projects/ai-agent-code-polisher/polished-code/sample_two.js
Rewriting /Users/huzaifa/projects/ai-agent-code-polisher/unoptimized-code/sample_one.py...
✅ Wrote rewritten file to /Users/huzaifa/projects/ai-agent-code-polisher/polished-code/sample_one.py
